데이터 설명: https://www.kaggle.com/uciml/iris
StandardScaler: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
import math

In [22]:
iris = load_iris()

In [23]:
#iris.data -> features, iris.target -> labels
X = iris.data
y = iris.target

In [39]:
# split into test and train dataset, and use random_state=48
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=50)

In [25]:
from sklearn.preprocessing import StandardScaler

In [40]:
ss = StandardScaler() #Standardize features by removing the mean and scaling to unit variance

X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)

In [41]:
trainSet=np.column_stack((X_train, y_train))
testSet=np.column_stack((X_test, y_test))

In [42]:
testSet

array([[-0.27340169, -0.19650869,  0.22079092,  0.15623102,  1.        ],
       [ 0.59651278, -1.35568919,  0.67872763,  0.42081581,  1.        ],
       [-0.52194868,  0.7308357 , -1.15301923, -1.29898533,  0.        ],
       [-0.52194868,  1.8900162 , -1.38198759, -1.03440054,  0.        ],
       [ 0.84505977, -0.19650869,  0.8504539 ,  1.08227779,  2.        ],
       [ 1.09360677,  0.03532741,  1.07942226,  1.61144737,  2.        ],
       [-0.1491282 , -1.35568919,  0.73596972,  1.08227779,  2.        ],
       [-0.89476917,  0.4989996 , -1.15301923, -0.90210814,  0.        ],
       [-1.26758966, -0.19650869, -1.3247455 , -1.43127773,  0.        ],
       [ 0.47223929, -2.05119749,  0.44975928,  0.42081581,  1.        ],
       [-0.89476917,  0.7308357 , -1.26750341, -1.29898533,  0.        ],
       [ 1.96352124, -0.66018089,  1.36563271,  0.94998539,  2.        ],
       [-1.14331617,  0.03532741, -1.26750341, -1.29898533,  0.        ],
       [ 1.21788026, -0.19650869,  1.0

### KNN Classifier Implementation

k개의 가장 가까운 사례를 사용하여 수치 예측 및 분류 수행

In [43]:
# 데이터 인스턴스 간 유사성을 계산함. 
def getDistance(p, instance1, instance2):
    distance = 0
    length = len(instance1)-1 # 맨 끝은 label
    
    # p=1 : return Manhattan Distance
    if p==1:
        for x in range(length):
            distance += abs(float(instance1[x]) - float(instance2[x]))
        
    # p=2 : return Eucludean Distance
    elif p==2:
        for x in range(length):
            distance += (float(instance1[x]) - float(instance2[x]))**2
        distance = math.sqrt(distance)
    
    return distance

In [44]:
import operator

# 주어진 testInstance에 대한 trainSet로부터 가장 유사한 neighbors를 k 개 반환

def getNeighbors(p, trainSet, testInstance, k):
    distances = []
    
    # p=1 : return Manhattan Distance
    if p==1:
        for x in range(len(trainSet)):
            distance = getDistance(1, testInstance, trainSet[x])
            distances.append((trainSet[x], distance))
        
    # p=2 : return Eucludean Distance
    elif p==2:
        for x in range(len(trainSet)):
            distance = getDistance(2, testInstance, trainSet[x])
            distances.append((trainSet[x], distance))
            
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
        
    return neighbors

In [45]:
# 생성된 neighbors 기반으로 response를 예측하고, 다수결을 알려준다.

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    vote = sortedVotes[0][0]
    return vote

In [46]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    accuracy_score = correct/float(len(testSet)) * 100.0
    return accuracy_score

In [47]:
k = 3
p = 2 # Euclidean distance

In [48]:
predictions=[]

for i in range(len(testSet)):
    neighbors = getNeighbors(p, trainSet, testSet[i], k)
    result = getResponse(neighbors)
    predictions.append(result)
    print(str(i) + ' > predicted : ' + str(result) + ', actual : ' + str(testSet[i][-1]))

0 > predicted : 1.0, actual : 1.0
1 > predicted : 2.0, actual : 1.0
2 > predicted : 0.0, actual : 0.0
3 > predicted : 0.0, actual : 0.0
4 > predicted : 2.0, actual : 2.0
5 > predicted : 2.0, actual : 2.0
6 > predicted : 2.0, actual : 2.0
7 > predicted : 0.0, actual : 0.0
8 > predicted : 0.0, actual : 0.0
9 > predicted : 1.0, actual : 1.0
10 > predicted : 0.0, actual : 0.0
11 > predicted : 2.0, actual : 2.0
12 > predicted : 0.0, actual : 0.0
13 > predicted : 2.0, actual : 2.0
14 > predicted : 1.0, actual : 1.0
15 > predicted : 0.0, actual : 0.0
16 > predicted : 1.0, actual : 1.0
17 > predicted : 0.0, actual : 0.0
18 > predicted : 1.0, actual : 1.0
19 > predicted : 1.0, actual : 1.0
20 > predicted : 2.0, actual : 2.0
21 > predicted : 1.0, actual : 1.0
22 > predicted : 0.0, actual : 0.0
23 > predicted : 2.0, actual : 2.0
24 > predicted : 1.0, actual : 1.0
25 > predicted : 2.0, actual : 2.0
26 > predicted : 1.0, actual : 1.0
27 > predicted : 1.0, actual : 1.0
28 > predicted : 1.0, actual :

In [49]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + str(accuracy) + '%')

Accuracy: 95.55555555555556%
